In [2]:
from pymongo import MongoClient

import jieba
import jieba.analyse

from datetime import datetime, timedelta
import pprint

import numpy as np
import pandas as pd

from collections import OrderedDict

import numpy as np

import bokeh
from bokeh.layouts import gridplot
from bokeh.plotting import *
# from bokeh.sampledata.stocks import AAPL
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook

output_notebook(hide_banner=False)

jieba.analyse.set_stop_words('data/stopwords-zh.txt')

Loading BokehJS ...

In [3]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client.newsdata
news = db.news

In [6]:
trivial = ['\n', '\t', '\xa0', '\u3000', ' ', '·', '"', '—', ':', ',', '=', '’', '…', ';', "'", '+', '..', '?', '＂', '－', 'Ｘ', ']', '[']

f = open('data/stopwords-zh.txt', 'r').read()

stop_words = f.split()
stop_words += trivial

In [7]:
class EventDate:
    def __init__(self, date, before, after):
        self.event_date  = date
        self.first_date  = datetime(2013, 4, 24)
        self.last_date   = datetime(2016, 8, 24)
        self.start_point = date - timedelta(days = before)
        self.end_point   = date + timedelta(days = after+1, seconds=-1)
        
#     get all_data, all_content, news_amount
    def get_data(self):
        self.all_data    = {}
        self.all_content = ""
        
        for item in news.find({'post_time': {'$gte': self.start_point, '$lt': self.end_point}}):
            if "视频" in item['title']:
                continue
            
            date = item['post_time']
            date = datetime(date.year, date.month, date.day)
            
            self.all_content += item['content']
            
            if date in self.all_data:
                self.all_data[date].append(item)
            else:
                self.all_data[date] = [item]
                
        self.news_amount = sum( [len(i) for i in list(self.all_data.values())] )
        
#     get words_amount, tf, ordered_tf, word_order
    def get_tf(self):
        count = {}
        
        words = jieba.cut(self.all_content)
        
        for word in words:
            if word not in stop_words:
                if word in count:
                    count[word] += 1
                else:
                    count[word]  = 1
        
        self.words_amount = sum(count.values())
        
        count = {k: v/total for total in (self.words_amount, ) for k, v in count.items()}

        self.tf         = count
        self.ordered_tf = OrderedDict(sorted(count.items(), key= lambda t: t[1], reverse= True))
        self.word_order = list(self.ordered_tf.keys())

In [9]:
date1 = datetime(2014, 3, 27)
date2 = datetime(2014, 4, 10)
date3 = datetime(2015, 1, 14)

e1 = EventDate(date1, 4, 4)
e2 = EventDate(date2, 4, 4)
e3 = EventDate(date3, 4, 4)

e1.get_data()
e1.get_tf()
e2.get_data()
e2.get_tf()
e3.get_data()
e3.get_tf()

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 2.391 seconds.
Prefix dict has been built succesfully.


In [11]:
# high_frq_of_3 = list(set(e1.word_order[:2500]) | set(e2.word_order[:2500]) | set(e3.word_order[:2500]))
high_frq_of_3 = list( set(e1.word_order[:2500]) & set(e2.word_order[:2500]) & set(e3.word_order[:2500]) )

In [12]:
d = {}
# for i in high_frq_of_3:
#     d[i] = []
#     if i in e1.tf:
#         d[i].append(e1.tf[i])
#     else:
#         d[i].append(0.0)
#     if i in e2.tf:
#         d[i].append(e2.tf[i])
#     else:
#         d[i].append(0.0)
#     if i in e3.tf:
#         d[i].append(e3.tf[i])
#     else:
#         d[i].append(0.0)
for i in high_frq_of_3:
    d[i] = [e1.tf[i], e2.tf[i], e3.tf[i]]
    
std = {}
for i in high_frq_of_3:
    std[i] = np.std([e1.tf[i], e2.tf[i], e3.tf[i]])

In [14]:
# only appear in either e1 or e2
a = list(set(e1.word_order) ^ set(e2.word_order))

d = []
for i in a:
    try:
        if e1.word_order.index(i)<200:
            print('e1', i, e1.word_order.index(i)) 
    except:
        if e2.word_order.index(i)<200:
            print('e2', i, e2.word_order.index(i))
# for i in e2.ordered_tf
# print(e2.word_order.index('GPU'))

e1 芬尼 73
e2 22196 117
e2 22227.1 74
e2 巴西 164
e2 课程 190
e1 22199.1 199
e1 李笑 141


In [11]:
tf_of_3 = {
    'e1' : pd.Series(e1.tf),
    'e2' : pd.Series(e2.tf),
    'e3' : pd.Series(e3.tf), 
    'std': pd.Series(std)
}
df = pd.DataFrame(tf_of_3, index = high_frq_of_3)
df

,e1,e2,e3,std
一项,0.000246,0.000164,0.000521,0.000153
数学,0.000074,0.000328,0.000130,0.000109
Bitcoin,0.000491,0.000683,0.000261,0.000173
汇款,0.000467,0.000492,0.000347,0.000063
买价,0.002456,0.003143,0.002780,0.000281
小时,0.000491,0.000273,0.000261,0.000106
CEO,0.000491,0.000683,0.000565,0.000079
CoinDesk,0.000639,0.000137,0.000782,0.000277
持有者,0.000123,0.000164,0.000087,0.000032
市场,0.002480,0.002159,0.004430,0.001004
